In [1]:
def make_dict(sub):
    check_dict[sub] = {}

In [ ]:
import os, shutil, glob
import pandas as pd 

#BIDS_PATH = 
#MULTI_SESS =

#def check_subjects():
## Set input path for BIDS data and get subjects 
SUBJ_DIRS = glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/Bevel/BIDS", "sub-*"))
check_dict = {} ## initialize dictionary


## iterate through subjects and inspect their files and 
for SUBDIR in sorted(SUBJ_DIRS):
    sub_id = SUBDIR.split("/")[-1] # get subject id
    # MAKE DICTIONARY 
    check_dict[sub_id] = {} # add subject id to dictionary
    # GET RELEVANT PATHS, AND FILES 
    ## --here we set the data directory paths we want to check : (func/, anat/, and fmap/)
    ## and then we grab all the *nii.gz files within them
    prob_files = glob.glob(os.path.join(SUBDIR, 'func', '*task-prob*'))
    rest_files = glob.glob(os.path.join(SUBDIR, 'func', '*task-rest*'))
    anat_files = glob.glob(os.path.join(SUBDIR, 'anat', '*T1w*'))
    magn_files = glob.glob(os.path.join(SUBDIR, 'fmap', '*magnitude*'))
    phase_files = glob.glob(os.path.join(SUBDIR, 'fmap', '*phasediff*'))

    # IDENTIFY ERRORS IN func/
    # I. Check "task-prob" files
    if not prob_files:
        ERROR = "NO functional files found"
    else:
        # for each run expect to have total of 3 files - *bold.json, *bold.nii.gz, *events.tsv
        run1_files = glob.glob(os.path.join(SUBDIR, "func", "*run-1*"))
        run2_files = glob.glob(os.path.join(SUBDIR, "func", "*run-2*"))
        run3_files = glob.glob(os.path.join(SUBDIR, "func", "*run-3*"))
        run4_files = glob.glob(os.path.join(SUBDIR, "func", "*run-4*"))
        runs = [run1_files, run2_files, run3_files, run4_files]
        keywordlist = [".tsv", ".json", "nii.gz"]
        run_lists = [run1_files, run2_files, run3_files, run4_files]
        target = 0
        for run in run_lists:
            #initialize local parameters
            tsv_check = False
            json_check = False
            nifti_check = False
            for file in run:
                for keyword in keywordlist:
                    if keyword in file:
                        #print("PASSED %s"%keyword)
                        if keyword == ".tsv":
                            tsv_check = True 
                        elif keyword == ".json":
                            json_check = True 
                        else:
                            nifti_check = True
            checklist=[tsv_check, json_check, nifti_check]
            if (tsv_check and json_check and nifti_check) == True:
                print("PASS")
        
                        

        func_check = "FAIL"
        #######WRITE TO FILE -- QUALITY ANALYSIS REPORTING
    else: # IDENTIFY FUNCS
        train_run_count=0
        rl_count = 0 
        # Expecting 4 BEVEL functionals 
        for func_ in sorted(FUNCS):
            name = func_.split("/")[-1] 
            TASK = name.split("_")[1]
            print(name)
            if "rest" in name:
                resting_file = name
                rest_check = "PASS"
                check_dict[SUBJECT]["REST_CHECK"] = rest_check 
            else:
                if "prob" in TASK:
                    if "run" in name:
                        TRAIN_RUN = name.split("_")[2]
                        train_run_count = train_run_count + 1
        if train_run_count == 4:
            func_check = "PASS"
        else:
            func_check = "FAIL"
    check_dict[SUBJECT]["TRAIN_COUNT"] = train_run_count
    check_dict[SUBJECT]["FUNC_TRAIN_CHECK"] = func_check
    
            #rint("RUN COUNT: ", run_count)
    # IDENTIFY ANATS AND ANY ERRORS
    if not ANATS:
        anat_check = "FAIL"
        #print("********ERROR! NO ANATOMICALS: ", SUBJECT)
        #######WRITE TO FILE -- QUALITY ANALYSIS REPORTING
    else:
        anat_file = str(ANATS)
        anat_check = "PASS"
    check_dict[SUBJECT]["ANAT_CHECK"] = anat_check
    # IDENTIFY FMAPS AND ANY ERRORS
    ## here I am initializing the check variables
    phase_check = "FAIL"
    mag1_check = "FAIL"
    mag2_check = "FAIL"
    if not FMAPS:
        print("********ERROR! NO FMAPS: ", SUBJECT)
        fmap_check = "FAIL"
    else:
        for file in FMAPS:
            name = file.split("/")[-1]
            IDENTIFIER = name.split("_")[1]
            if "phasediff" in IDENTIFIER:
                phase_check = "PASS"
                phase_file = file
            if "magnitude1" in IDENTIFIER:
                mag1_check = "PASS"
                mag1_file = file
            if "magnitude2" in IDENTIFIER: 
                mag2_check = "PASS"
                mag2_file = file
        #print(phase_check, mag1_check, mag2_check)
        if phase_check and mag1_check and mag2_check == "PASS":
            fmap_check = "PASS"
        elif phase_check and mag1_check and mag2_check != "PASS":
            fmap_check = "FAIL"
        else:
            fmap_check = "FAIL"
    check_dict[SUBJECT]["FMAP_CHECK"] = fmap_check
